In [1]:
import pandas as pd
from soccerapi.api import Api888Sport
# from soccerapi.api import ApiUnibet
# from soccerapi.api import ApiBet365

api = Api888Sport()
url = 'https://www.888sport.com/#/filter/football/italy/serie_a'
odds = api.odds(url)

predictions_link = 'C:/Users/99451/Desktop/MODEL/2025/2024-10-17.xlsx'

pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Use json_normalize to flatten nested dictionaries
df = pd.json_normalize(odds)
competitions = api.competitions()
competitions_df = pd.json_normalize(competitions)

# Transpose the DataFrame
competitions_df_transposed = competitions_df.T

# Reset index to turn the index into a column
competitions_df_transposed.reset_index(inplace=True)

competitions_df_transposed.columns = ['League', 'Links']

# Display the DataFrame
competitions_df_transposed.head()


,League,Links
0,UEFA Nations League.UEFA Nations League,https://www.888sport.com/#/filter/football/uef...
1,Italy.Serie A,https://www.888sport.com/#/filter/football/ita...
2,Italy.Serie B,https://www.888sport.com/#/filter/football/ita...
3,Italy.Coppa Italia,https://www.888sport.com/#/filter/football/ita...
4,England.Premier League,https://www.888sport.com/#/filter/football/eng...


In [2]:
matches = pd.DataFrame()

for i in range(len(competitions_df_transposed)):
    url = competitions_df_transposed['Links'].iloc[i]
    odds = api.odds(url)
    my_df = pd.json_normalize(odds)
    matches = pd.concat([matches, my_df], ignore_index = True)

# Automatically select all numeric columns and divide by 1000
numeric_cols = matches.select_dtypes(include='number').columns
matches[numeric_cols] = matches[numeric_cols] / 1000
matches.head()

,time,home_team,away_team,full_time_result.1,full_time_result.X,full_time_result.2,under_over.O2.5,under_over.U2.5,both_teams_to_score.yes,both_teams_to_score.no,double_chance.1X,double_chance.12,double_chance.2X,both_teams_to_score,under_over,double_chance,full_time_result
0,2024-10-15T18:45:00Z,Switzerland,Denmark,2.23,3.2,3.30,2.35,1.56,1.96,1.74,1.32,1.33,1.61,NaN,NaN,NaN,NaN
1,2024-10-15T18:45:00Z,Lithuania,Romania,6.10,3.8,1.57,2.08,1.72,2.04,1.71,2.35,1.25,1.11,NaN,NaN,NaN,NaN
2,2024-10-15T18:45:00Z,Kosovo,Cyprus,1.33,4.9,9.50,1.77,2.00,2.10,1.67,1.03,1.15,3.20,NaN,NaN,NaN,NaN
3,2024-10-15T18:45:00Z,Spain,Serbia,1.33,5.0,9.00,1.67,2.15,1.95,1.75,1.07,1.16,3.05,NaN,NaN,NaN,NaN
4,2024-10-15T18:45:00Z,Northern Ireland,Bulgaria,1.88,3.1,4.70,1.55,2.40,2.38,1.53,1.16,1.33,1.86,NaN,NaN,NaN,NaN


In [3]:
predictions = pd.read_excel(predictions_link)
predictions['2.5U'] = 100 - predictions['2.5O']
predictions['OTTS'] = 100 - predictions['BTTS']

selected_columns = ['League','Home','Away','FT1', 'FTX', 'FT2', '2.5O', '2.5U','BTTS', 'OTTS', 'DC1X', 'DC12', 'DCX2']
predictions = predictions[selected_columns]

print('Number of matches predicted: ', len(predictions))
predictions.head()

Number of matches predicted:  3


,League,Home,Away,FT1,FTX,FT2,2.5O,2.5U,BTTS,OTTS,DC1X,DC12,DCX2
0,Brazil,Fortaleza,Atletico MG,59.13,22.62,18.23,55.30,44.70,53.51,46.49,81.75,77.36,40.85
1,Brazil,Sao Paulo,Vasco da Gama,56.29,24.49,19.21,49.18,50.82,49.61,50.39,80.78,75.50,43.70
2,England3,Shrewsbury,Exeter City,18.06,30.03,51.91,33.00,67.00,37.33,62.67,48.09,69.97,81.94


In [4]:
import pandas as pd
from fuzzywuzzy import process

#Renaming matches column to match with predictions
matches = matches.rename(columns={'home_team': 'HomeTeam', 'away_team': 'AwayTeam'})

# Function to match similar strings
def fuzzy_merge(df1, df2, key1, key2, threshold=75, limit=1):
    """
    df1, df2: DataFrames to merge
    key1, key2: Column names on which to perform the fuzzy match
    threshold: Similarity threshold (0-100), the higher the stricter
    limit: Maximum matches to return
    """
    s = df2[key2].tolist()

    # Create a new DataFrame to store the best match
    matches = df1[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))
    
    # Extract matched values and merge
    df1['Best_Match'] = matches.apply(lambda x: x[0] if x else None)
    df1['Match_Score'] = matches.apply(lambda x: x[1] if x else None)
    
    # Merge on the best match
    return pd.merge(df1, df2, left_on='Best_Match', right_on=key2)

# Use the fuzzy merge function
result = fuzzy_merge(predictions, matches, 'Home', 'HomeTeam')
result.drop(['both_teams_to_score', 'under_over', 'double_chance', 'full_time_result'], axis=1, inplace = True)
print('Number of matches found with odds: {} number of matches missing: {}'.format(len(result), len(predictions) - len(result)))
result.head()


Number of matches found with odds: 3 number of matches missing: 0


,League,Home,Away,FT1,FTX,FT2,2.5O,2.5U,BTTS,OTTS,DC1X,DC12,DCX2,Best_Match,Match_Score,time,HomeTeam,AwayTeam,full_time_result.1,full_time_result.X,full_time_result.2,under_over.O2.5,under_over.U2.5,both_teams_to_score.yes,both_teams_to_score.no,double_chance.1X,double_chance.12,double_chance.2X
0,Brazil,Fortaleza,Atletico MG,59.13,22.62,18.23,55.30,44.70,53.51,46.49,81.75,77.36,40.85,Fortaleza-CE,95,2024-10-17T00:45:00Z,Fortaleza-CE,Atlético Mineiro-MG,1.75,3.60,4.6,2.20,1.65,1.98,1.73,1.18,1.26,1.96
1,Brazil,Sao Paulo,Vasco da Gama,56.29,24.49,19.21,49.18,50.82,49.61,50.39,80.78,75.50,43.70,São Paulo-SP,80,2024-10-17T00:45:00Z,São Paulo-SP,Vasco da Gama-RJ,1.68,3.65,5.1,2.32,1.58,2.12,1.64,1.15,1.26,2.05
2,England3,Shrewsbury,Exeter City,18.06,30.03,51.91,33.00,67.00,37.33,62.67,48.09,69.97,81.94,Shrewsbury Town,90,2024-10-17T19:00:00Z,Shrewsbury Town,Exeter,2.85,3.30,2.3,2.07,1.70,1.81,1.88,1.57,1.30,1.40


# Calculating probabilities and finding value bets

In [5]:
ftrs = []
ftrs_string = []
ftrs_percent = []
ftrs_odds = []
overs = []
overs_string = []
overs_percent = []
overs_odds = []
btts = []
btts_string = []
btts_percent = []
btts_odds = []
dcs = []
dcs_string = []
dcs_percent = []
dcs_odds = []

for i in range(len(result)):
    if result['DC1X'].iloc[i] > result['DC12'].iloc[i] and result['DC1X'].iloc[i] > result['DCX2'].iloc[i]:
        dcs_string.append('DC1X')
        dcs_odds.append(result['double_chance.1X'].iloc[i])
        dcs_percent.append(result['DC1X'].iloc[i])
        value = result['double_chance.1X'].iloc[i] - (1 / (result['DC1X'].iloc[i] / 100))
        dcs.append(round(value,2))
    elif result['DCX2'].iloc[i] > result['DC1X'].iloc[i] and result['DCX2'].iloc[i] > result['DC12'].iloc[i]:
        dcs_string.append('DCX2')
        dcs_odds.append(result['double_chance.2X'].iloc[i])
        dcs_percent.append(result['DCX2'].iloc[i])
        value = result['double_chance.2X'].iloc[i] - (1 / (result['DCX2'].iloc[i] / 100))
        dcs.append(round(value,2))
    else:
        dcs_string.append('DC12')
        dcs_odds.append(result['double_chance.12'].iloc[i])
        dcs_percent.append(result['DC12'].iloc[i])
        value = result['double_chance.12'].iloc[i] - (1 / (result['DC12'].iloc[i] / 100))
        dcs.append(round(value,2))

    if result['FT1'].iloc[i] > result['FTX'].iloc[i] and result['FT1'].iloc[i] > result['FT2'].iloc[i]:
        ftrs_string.append('FT1')
        ftrs_odds.append(result['full_time_result.1'].iloc[i])
        ftrs_percent.append(result['FT1'].iloc[i])
        value = result['full_time_result.1'].iloc[i] - (1 / (result['FT1'].iloc[i] / 100))
        ftrs.append(round(value,2))
    elif result['FTX'].iloc[i] > result['FT1'].iloc[i] and result['FTX'].iloc[i] > result['FT2'].iloc[i]:
        ftrs_string.append('FTX')
        ftrs_odds.append(result['full_time_result.X'].iloc[i])
        ftrs_percent.append(result['FTX'].iloc[i])
        value = result['full_time_result.X'].iloc[i] - (1 / (result['FTX'].iloc[i] / 100))
        ftrs.append(round(value,2))
    else:
        ftrs_string.append('FT2')
        ftrs_odds.append(result['full_time_result.2'].iloc[i])
        ftrs_percent.append(result['FT2'].iloc[i])
        value = result['full_time_result.2'].iloc[i] - (1 / (result['FT2'].iloc[i] / 100))
        ftrs.append(round(value,2))
    
    if result['2.5O'].iloc[i] > result['2.5U'].iloc[i]:
        overs_string.append('2.5O')
        overs_odds.append(result['under_over.O2.5'].iloc[i])
        overs_percent.append(result['2.5O'].iloc[i])
        value = result['under_over.O2.5'].iloc[i] - (1 / (result['2.5O'].iloc[i] / 100))
        overs.append(round(value, 2))
    else:
        overs_string.append('2.5U')
        overs_odds.append(result['under_over.U2.5'].iloc[i])
        overs_percent.append(result['2.5U'].iloc[i])
        value = result['under_over.U2.5'].iloc[i] - (1 / (result['2.5U'].iloc[i] / 100))
        overs.append(round(value, 2))

    if result['BTTS'].iloc[i] > result['OTTS'].iloc[i]:
        btts_string.append('BTTS')
        btts_odds.append(result['both_teams_to_score.yes'].iloc[i])
        btts_percent.append(result['BTTS'].iloc[i])
        value = result['both_teams_to_score.yes'].iloc[i] - (1 / (result['BTTS'].iloc[i] / 100))
        btts.append(round(value, 2))
    else:
        btts_string.append('OTTS')
        btts_odds.append(result['both_teams_to_score.no'].iloc[i])
        btts_percent.append(result['OTTS'].iloc[i])
        value = result['both_teams_to_score.no'].iloc[i] - (1 / (result['OTTS'].iloc[i] / 100))
        btts.append(round(value, 2))

    
# Create a DataFrame
df = pd.DataFrame({
    'League': result['League'],
    'HomeTeam': result['HomeTeam'],
    'AwayTeam': result['AwayTeam'],
    'DCBets': dcs_string,
    'DCPercent': dcs_percent,
    'DCOdd': dcs_odds,
    'DCVal': dcs,
    'FTRBets': ftrs_string,
    'FTRPercent': ftrs_percent,
    'FTROdd': ftrs_odds,
    'FTRVal': ftrs,
    '2.5O/U': overs_string,
    '2.5Percent': overs_percent,
    '2.5Odd': overs_odds,
    '2.5Val': overs,
    'BTTS': btts_string,
    'BTPercent': btts_percent,
    'BTOdd': btts_odds,
    'BTVal': btts
})

#Adding recommended bets according to value difference
rec_bet, rec_per, rec_odd, rec_val = [], [], [], []

for i in range(len(df)):
    values = [df['DCVal'].iloc[i], df['FTRVal'].iloc[i], df['2.5Val'].iloc[i], df['BTVal'].iloc[i]]
    if max(values) == df['DCVal'].iloc[i]:
        rec_bet.append(df['DCBets'].iloc[i])
        rec_per.append(df['DCPercent'].iloc[i])
        rec_odd.append(df['DCOdd'].iloc[i])
        rec_val.append(df['DCVal'].iloc[i])
    elif max(values) == df['FTRVal'].iloc[i]:
        rec_bet.append(df['FTRBets'].iloc[i])
        rec_per.append(df['FTRPercent'].iloc[i])
        rec_odd.append(df['FTROdd'].iloc[i])
        rec_val.append(df['FTRVal'].iloc[i])
    elif max(values) == df['2.5Val'].iloc[i]:
        rec_bet.append(df['2.5O/U'].iloc[i])
        rec_per.append(df['2.5Percent'].iloc[i])
        rec_odd.append(df['2.5Odd'].iloc[i])
        rec_val.append(df['2.5Val'].iloc[i])
    else:
        rec_bet.append(df['BTTS'].iloc[i])
        rec_per.append(df['BTPercent'].iloc[i])
        rec_odd.append(df['BTOdd'].iloc[i])
        rec_val.append(df['BTVal'].iloc[i])

rec_df = pd.DataFrame({
    'Rec': rec_bet,
    'RecPer': rec_per,
    'RecOdd': rec_odd,
    'RecVal': rec_val
})

# Concatenate the new DataFrame with the existing one
df = pd.concat([rec_df, df], axis=1)

first_three_columns = ['League', 'HomeTeam', 'AwayTeam']
new_columns = ['Rec', 'RecPer', 'RecOdd', 'RecVal']
remaining_columns = [col for col in df.columns if col not in (first_three_columns + new_columns)]
# Rearrange the columns
df = df[first_three_columns + new_columns + remaining_columns]

# Function to highlight values higher than threshold
def highlight_values(value):
    if isinstance(value, str) or pd.isna(value):
        return ''  # Return empty string for NaN values
    elif value > 80:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = df.style.applymap(highlight_values)
styled_df.to_excel('final_games.xlsx', index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

C:\Users\99451\AppData\Local\Temp\ipykernel_17432\1761804651.py:161: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = df.style.applymap(highlight_values)


,League,HomeTeam,AwayTeam,Rec,RecPer,RecOdd,RecVal,DCBets,DCPercent,DCOdd,DCVal,FTRBets,FTRPercent,FTROdd,FTRVal,2.5O/U,2.5Percent,2.5Odd,2.5Val,BTTS,BTPercent,BTOdd,BTVal
0,Brazil,Fortaleza-CE,Atlético Mineiro-MG,2.5O,55.300000,2.200000,0.390000,DC1X,81.750000,1.180000,-0.040000,FT1,59.130000,1.750000,0.060000,2.5O,55.300000,2.200000,0.390000,BTTS,53.510000,1.980000,0.110000
1,Brazil,São Paulo-SP,Vasco da Gama-RJ,DC1X,80.780000,1.150000,-0.090000,DC1X,80.780000,1.150000,-0.090000,FT1,56.290000,1.680000,-0.100000,2.5U,50.820000,1.580000,-0.390000,OTTS,50.390000,1.640000,-0.340000
2,England3,Shrewsbury Town,Exeter,FT2,51.910000,2.300000,0.370000,DCX2,81.940000,1.400000,0.180000,FT2,51.910000,2.300000,0.370000,2.5U,67.000000,1.700000,0.210000,OTTS,62.670000,1.880000,0.280000
